# 🧠 RAG Agent with `inference-matcher-beeai`



In [ ]:
##  Full Notebook Example of Usage
# -----------------------------------------------------------
# 🔧 Setup
# -----------------------------------------------------------
# Install all required packages. Run this cell only once.
#!pip install -r ../requirements.txt
#!pip install rouge-score nltk pandas streamlit


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached streamlit-1.44.1-py3-none-any.whl.metadata (8.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 71.2 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.2 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24986 sha256=2056d53cd2adc6e1aad9a898197cd980

In [1]:
# -----------------------------------------------------------
# 🧪 Step 1: Data Cleaning
# -----------------------------------------------------------

import pandas as pd
import re
import os

In [2]:
# Ensure we can import from parent src/ directory
import sys
sys.path.append(os.path.abspath(".."))

# Load raw dataset
df = pd.read_csv("./data/rag_samples.csv")

In [3]:
df

,Unnamed: 0,ki_topic,ki_text,sample_question,sample_ground_truth,sample_answer
0,0,Setting Up a Mobile Device for Company Email,**Setting Up a Mobile Device for Company Email...,"""How do I set up my company email on my mobile...",To set up your company email on your mobile de...,To set up your company email on your mobile d...
1,1,Resetting a Forgotten PIN,**Resetting a Forgotten PIN**\n\nIf you have f...,"I forgot my PIN, how can I reset it?","Don't worry, I'm here to help To reset your fo...","To reset your forgotten PIN, follow these ste..."
2,2,Configuring VPN Access for Remote Workers,**Configuring VPN Access for Remote Workers**\...,How do I set up VPN access on my laptop so I c...,To set up VPN access on your laptop and access...,"To set up VPN access on your laptop, follow t..."
3,3,Troubleshooting Issues with Microsoft Office,**Troubleshooting Issues with Microsoft Office...,"""My Microsoft Word keeps freezing every time I...",I'd be happy to help you troubleshoot the issu...,"Try restarting Microsoft Word, then check for..."
4,4,Setting Up a Conference Call on Cisco Webex,"To set up a conference call on Cisco Webex, fo...",How do I set up a conference call on Cisco Web...,To set up a conference call on Cisco Webex wit...,To set up a conference call on Cisco Webex wi...
5,5,Creating a Backup of Important Files,**Creating a Backup of Important Files**\n\nBa...,"""How do I back up my important work files to p...","To back up your important work files, follow t...","To back up your important work files, follow ..."
6,6,Troubleshooting Issues with Company-Issued Tab...,**Troubleshooting Issues with Company-Issued T...,"""My company-issued tablet is freezing frequent...",I'd be happy to help you troubleshoot the issu...,To fix the issue with your company-issued tab...
7,7,Setting Up a Secure Wireless Network,**Step 1: Plan Your Wireless Network**\n\nBefo...,"""How do I set up a secure wireless network for...",To set up a secure wireless network for your w...,"To set up a secure wireless network, follow t..."
8,8,Resetting a Jammed Printer,**Resetting a Jammed Printer**\n\n**Step 1: Tu...,What steps can I take to fix my printer when i...,"Don't worry, I'm here to help To fix a jammed ...","To fix a jammed printer, follow these steps: ..."
9,9,Configuring Email on an Android Device,**Configuring Email on an Android Device**\n\n...,How do I set up my company email on my persona...,To set up your company email on your personal ...,To set up your company email on your personal...


In [4]:
# Cleaning function
def clean_text(text):
    text = re.sub(r'http\S+', '', text)                            # remove URLs
    text = re.sub(r'\b\w*\d\w*\b', '', text)                       # remove alphanumerics
    text = re.sub(r'#\S+', '', text)                               # remove hashtags
    text = re.sub(r'<.*?>', '', text)                              # remove HTML tags
    return text.strip()

In [5]:
# Apply cleaning
df["cleaned_ki_text"] = df["ki_text"].apply(clean_text)
df.to_csv("./data/rag_cleaned.csv", index=False)
df.head()

,Unnamed: 0,ki_topic,ki_text,sample_question,sample_ground_truth,sample_answer,cleaned_ki_text
0,0,Setting Up a Mobile Device for Company Email,**Setting Up a Mobile Device for Company Email...,"""How do I set up my company email on my mobile...",To set up your company email on your mobile de...,To set up your company email on your mobile d...,**Setting Up a Mobile Device for Company Email...
1,1,Resetting a Forgotten PIN,**Resetting a Forgotten PIN**\n\nIf you have f...,"I forgot my PIN, how can I reset it?","Don't worry, I'm here to help To reset your fo...","To reset your forgotten PIN, follow these ste...",**Resetting a Forgotten PIN**\n\nIf you have f...
2,2,Configuring VPN Access for Remote Workers,**Configuring VPN Access for Remote Workers**\...,How do I set up VPN access on my laptop so I c...,To set up VPN access on your laptop and access...,"To set up VPN access on your laptop, follow t...",**Configuring VPN Access for Remote Workers**\...
3,3,Troubleshooting Issues with Microsoft Office,**Troubleshooting Issues with Microsoft Office...,"""My Microsoft Word keeps freezing every time I...",I'd be happy to help you troubleshoot the issu...,"Try restarting Microsoft Word, then check for...",**Troubleshooting Issues with Microsoft Office...
4,4,Setting Up a Conference Call on Cisco Webex,"To set up a conference call on Cisco Webex, fo...",How do I set up a conference call on Cisco Web...,To set up a conference call on Cisco Webex wit...,To set up a conference call on Cisco Webex wi...,"To set up a conference call on Cisco Webex, fo..."


In [20]:
# -----------------------------------------------------------
# 📥 Step 2: Load Prompt Templates & Setup Agent Inputs
# -----------------------------------------------------------

from src.templates.agent_generation_template import agent_generation_template
from src.evaluation.evaluation_utils import calculate_similarity, meets_criteria
from src.workflows.inference_matcher_workflow import inference_workflow, InferenceMatcherState
import json
import asyncio
# Join cleaned content into one string for prompt base
document_template = "\n".join(df["cleaned_ki_text"].tolist())
# Search parameters for different model or method settings
parameters = [
    "use_embedding=True, model=watsonx/granite-8b",
    "use_embedding=False, model=watsonx/codellama",
    "retriever_method=cosine"
]

# Define what our target agent should produce
target_config = {
    "target_description": "The generated inference should represent a valid configuration for a RAG support agent.",
    "criteria": {
        "context_usage": "true",
        "conciseness": "high",
        "retrieval_citation": "true"
    }
}

In [ ]:
# -----------------------------------------------------------
# 🚀 Step 3: Run Inference Matcher Workflow
# -----------------------------------------------------------

# Initialize agent state for BeeAI workflow
state = InferenceMatcherState(
    document_template=document_template,
    target_output=target_config,
    search_parameters=parameters,
    max_iterations=1
)


In [14]:
import nest_asyncio
import asyncio

# Patch the existing running loop
nest_asyncio.apply()

# Run the async workflow properly
result = await inference_workflow.run(state)


INFO:src.workflows.inference_matcher_workflow:[Step: Generate] Using Parameters: use_embedding=True, model=watsonx/granite-8b
13:16:49 - LiteLLM:INFO: utils.py:3075 - 
LiteLLM completion() model= llama3.1; provider = ollama_chat
INFO:LiteLLM:
LiteLLM completion() model= llama3.1; provider = ollama_chat



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



TypeError: unhashable type: 'list'

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
# Print final result
print("✅ Best Inference:")
print(result.state.best_inference)
print("💡 Score:", result.state.best_score)


In [ ]:
# -----------------------------------------------------------
# 📊 Step 4: Evaluation (ROUGE & Custom Relevance)
# -----------------------------------------------------------

from rouge_score import rouge_scorer
from difflib import SequenceMatcher

# Initialize scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

scores = []

# Evaluate each AI-generated answer against ground truth
for i, row in df.iterrows():
    rouge = scorer.score(row["sample_ground_truth"], row["sample_answer"])
    rel = SequenceMatcher(None, row["sample_ground_truth"], row["sample_answer"]).ratio()
    scores.append({
        "index": i,
        "ROUGE-L": round(rouge["rougeL"].fmeasure, 4),
        "Relevance": round(rel, 4)
    })

# Show results
eval_df = pd.DataFrame(scores)
eval_df.style.bar(subset=["ROUGE-L", "Relevance"], color='#5fba7d')

In [ ]:
# -----------------------------------------------------------
# 🔁 Optional: Launch the Streamlit Debugger from CLI
# -----------------------------------------------------------

# To use the debugging assistant with WatsonX, run from terminal:
# !streamlit run ../src/debugger/debugger_app.py